In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import cohen_kappa_score

In [2]:
df = pd.read_csv("processed_annotations/CANNONICAL_LIST.csv")
df.head()

,Unnamed: 0,ICD_doc_id,sentence,num_words,num_chars,sentence_string,KATHLEEN_A,KATHLEEN_B,KATHLEEN_text,annotation_id,...,KAYCEE_B,KAYCEE_text,LIZ_A,LIZ_B,LIZ_text,inclusion,Sum_Annotations,ANY_A,ANY_B,EITHER_AB
0,0,95581557,"Ct) MERCY HEALTH \nGRAND RAPIDS, Ml \nCONSENT ...",23,106,"Ct) MERCY HEALTH GRAND RAPIDS, Ml CONSENT TO R...",0,0,NaN,NaN,...,0,NaN,0,0,NaN,1,0,0,0,0
1,1,95581557,AND/OR BLOOD PRODUCTS \n,6,23,AND/OR BLOOD PRODUCTS,0,0,NaN,NaN,...,0,NaN,0,0,NaN,0,0,0,0,0
2,2,95581557,I understand that I need or may need blood and...,27,130,I understand that I need or may need blood and...,0,0,NaN,71891838.0,...,0,I understand that I need or may need blood and...,0,0,NaN,1,1,1,0,1
3,3,95581557,This hospital admission D Outpatient: Series o...,12,69,This hospital admission D Outpatient: Series o...,0,0,NaN,71891838.0,...,0,This hospital admission D Outpatient: Series o...,0,0,NaN,1,1,1,0,1
4,4,95581557,DESCRIPTION OF PROCEDURE,3,24,DESCRIPTION OF PROCEDURE,0,0,NaN,NaN,...,0,NaN,0,0,NaN,0,0,0,0,0


In [3]:
included = df[(df['inclusion'] == 1)]
annotated_AB = df[(df['EITHER_AB'] == 1)]
annotated_A = df[(df['ANY_A'] == 1)]
annotated_B = df[(df['ANY_B'] == 1)]

print(f"Total Sentences: {len(df)}")
print(f"Total Sentences (clean): {len(included)}; (% total: {len(included)/len(df):.3f})")

print(f"Annotated Either A or B: {len(annotated_AB)}; (% total: {len(annotated_AB)/len(df):.3f}); (% clean: {len(annotated_AB)/len(included):.3f})")
print(f"Annotated A: {len(annotated_A)}; (% total: {len(annotated_A)/len(df):.3f}); (% clean: {len(annotated_A)/len(included):.3f})")
print(f"Annotated B: {len(annotated_B)};(% total: {len(annotated_B)/len(df):.3f}); (% clean: {len(annotated_B)/len(included):.3f})")

Total Sentences: 9951
Total Sentences (clean): 6399; (% total: 0.643)
Annotated Either A or B: 739; (% total: 0.074); (% clean: 0.115)
Annotated A: 636; (% total: 0.064); (% clean: 0.099)
Annotated B: 139;(% total: 0.014); (% clean: 0.022)


In [4]:
N_docs = len(set(df['ICD_doc_id']))

KATHLEEN_A = np.sum(df['KATHLEEN_A'])
KATHLEEN_B = np.sum(df['KATHLEEN_B'])

LIZ_A = np.sum(df['LIZ_A'])
LIZ_B = np.sum(df['LIZ_B'])

KAYCEE_A = np.sum(df['KAYCEE_A'])
KAYCEE_B = np.sum(df['KAYCEE_B'])

print("Marked 'A'")
print(f"\tKATHLEEN: {KATHLEEN_A} ({KATHLEEN_A/N_docs:.3f} per doc)")
print(f"\tLIZ: {LIZ_A}  ({LIZ_A/N_docs:.3f} per doc)")
print(f"\tKAYCEE: {KAYCEE_A}  ({KAYCEE_A/N_docs:.3f} per doc)")
print()
print("Marked 'B'")
print(f"\tKATHLEEN: {KATHLEEN_B} ({KATHLEEN_B/N_docs:.3f} per doc)")
print(f"\tLIZ: {LIZ_B} ({LIZ_B/N_docs:.3f} per doc)")
print(f"\tKAYCEE: {KAYCEE_B} ({KAYCEE_B/N_docs:.3f} per doc)")

Marked 'A'
	KATHLEEN: 413 (3.082 per doc)
	LIZ: 333  (2.485 per doc)
	KAYCEE: 452  (3.373 per doc)

Marked 'B'
	KATHLEEN: 56 (0.418 per doc)
	LIZ: 59 (0.440 per doc)
	KAYCEE: 29 (0.216 per doc)


In [5]:
agree_A = included[(included['KATHLEEN_A'] == 1) & (included['KAYCEE_A'] == 1) & (included['LIZ_A'])]
agree_B = included[(included['KATHLEEN_B'] == 1) & (included['KAYCEE_B'] == 1) & (included['LIZ_B'])]
print(len(agree_A))
print(len(agree_B))

211
0


In [6]:
agree_any = included[((included['KATHLEEN_A'] == 1) | (included['KATHLEEN_B'] == 1) ) &\
                   ((included['LIZ_A'] == 1) | (included['LIZ_B'] == 1) ) &\
                   ((included['KAYCEE_A'] == 1) | (included['KAYCEE_B'] == 1) ) ]

len(agree_any)

231

In [7]:
blanks = included[((included['KATHLEEN_A'] == 0) & (included['KATHLEEN_B'] == 0) ) &\
                   ((included['LIZ_A'] == 0) & (included['LIZ_B'] == 0) ) &\
                   ((included['KAYCEE_A'] == 0) & (included['KAYCEE_B'] == 0) ) ]

len(blanks)

5660

In [8]:
5660/6399

0.8845132051883107

In [9]:
KATHLEENvLIZ_A = cohen_kappa_score(included['KATHLEEN_A'], included['LIZ_A'])
KATHLEENvKAYCEE_A = cohen_kappa_score(included['KATHLEEN_A'], included['KAYCEE_A'])
LIZvKAYCEE_A = cohen_kappa_score(included['LIZ_A'], included['KAYCEE_A'])

KATHLEENvLIZ_B = cohen_kappa_score(included['KATHLEEN_B'], included['LIZ_B'])
KATHLEENvKAYCEE_B = cohen_kappa_score(included['KATHLEEN_B'], included['KAYCEE_B'])
LIZvKAYCEE_B = cohen_kappa_score(included['KAYCEE_B'], included['LIZ_B'])

print("Marked 'A'")
print(f"KATHLEEN LIZ: {KATHLEENvLIZ_A:.4f}")
print(f"KATHLEEN KAYCEE: {KATHLEENvKAYCEE_A:.4f}")
print(f"LIZ KAYCEE : {LIZvKAYCEE_A:.4f}")
print()
print("Marked 'B'")
print(f"KATHLEEN LIZ: {KATHLEENvLIZ_B:.4f}")
print(f"KATHLEEN KAYCEE: {KATHLEENvKAYCEE_B:.4f}")
print(f"LIZ KAYCEE: {LIZvKAYCEE_B:.4f}")

Marked 'A'
KATHLEEN LIZ: 0.6131
KATHLEEN KAYCEE: 0.5921
LIZ KAYCEE : 0.6626

Marked 'B'
KATHLEEN LIZ: 0.0436
KATHLEEN KAYCEE: 0.0177
LIZ KAYCEE: 0.0168


In [10]:
# def fleiss_kappa(ratings, n):
#     '''
#     Computes the Fleiss' kappa measure for assessing the reliability of 
#     agreement between a fixed number n of raters when assigning categorical
#     ratings to a number of items.
    
#     Args:
#         ratings: a list of (item, category)-ratings
#         n: number of raters
#         k: number of categories
#     Returns:
#         the Fleiss' kappa score
    
#     See also:
#         http://en.wikipedia.org/wiki/Fleiss'_kappa
#     '''
#     items = set()
#     categories = set()
#     n_ij = {}
    
#     for i, c in ratings:
#         items.add(i)
#         categories.add(c)
#         n_ij[(i,c)] = n_ij.get((i,c), 0) + 1
    
#     N = len(items)
    
#     p_j = dict(((c, sum(n_ij.get((i, c), 0) for i in items) / (1.0 * n * N)) for c in categories))
#     P_i = dict(((i, (sum(n_ij.get((i, c), 0) ** 2 for c in categories) - n) / (n * (n - 1.0))) for i in items))

#     P_bar = sum(P_i.values()) / (1.0 * N)
#     P_e_bar = sum(value ** 2 for value in p_j.values())
    
#     kappa = (P_bar - P_e_bar) / (1 - P_e_bar)
    
#     return kappa

In [26]:
from statsmodels.stats.inter_rater import fleiss_kappa


included['sum_A'] = included['KATHLEEN_A'] + included['LIZ_A'] + included['KAYCEE_A']
included['sum_B'] = included['KATHLEEN_B'] + included['LIZ_B'] + included['KAYCEE_B']
included['sum_Blank'] = 3 - (included['sum_A'] + included['sum_B'])


fl = included[['sum_A', 'sum_B', 'sum_Blank']]

fleiss_kappa(fl, method='fleiss')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

0.5736460903453653

In [ ]:
KATHLEENvLIZ_A = cohen_kappa_score(included['KATHLEEN_A'] + 1, included['LIZ_A'] + 1)
KATHLEENvKAYCEE_A = cohen_kappa_score(included['KATHLEEN_A'] + 1, included['KAYCEE_A'] + 1)
LIZvKAYCEE_A = cohen_kappa_score(included['LIZ_A'] + 1, included['KAYCEE_A'] + 1)

KATHLEENvLIZ_B = cohen_kappa_score(included['KATHLEEN_B'] + 1, included['LIZ_B'] + 1)
KATHLEENvKAYCEE_B = cohen_kappa_score(included['KATHLEEN_B'] + 1, included['KAYCEE_B'] + 1)
LIZvKAYCEE_B = cohen_kappa_score(included['KAYCEE_B'] + 1, included['LIZ_B'] + 1)

print(f"KATHLEENvLIZ_A: {KATHLEENvLIZ_A:.4f}")
print(f"KATHLEENvKAYCEE_A: {KATHLEENvKAYCEE_A:.4f}")
print(f"LIZvKAYCEE_A: {LIZvKAYCEE_A:.4f}")
print(f"KATHLEENvLIZ_B: {KATHLEENvLIZ_B:.4f}")
print(f"KATHLEENvKAYCEE_B: {KATHLEENvKAYCEE_B:.4f}")
print(f"LIZvKAYCEE_B: {LIZvKAYCEE_B:.4f}")